In [25]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [26]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [27]:
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage
import operator

class AgentState(TypedDict):
  messages: Annotated[list[AnyMessage], operator.add]

In [35]:
from langchain_core.messages import ToolMessage, SystemMessage
from langgraph.graph import StateGraph, END

class Agent:
  def __init__(self, model, tools, system=""):
    self.system = system  # saving system message as attribute
    graph = StateGraph(AgentState)
    graph.add_node("llm", self.call_openai)
    graph.add_node("action", self.take_action)
    graph.add_conditional_edges(
      "llm",
      self.exists_action,
      {True: "action", False: END}  # depending on the function's return
    )
    graph.add_edge("action", "llm")
    graph.set_entry_point("llm")
    self.graph = graph.compile()
    self.tools = {t.name: t for t in tools}
    self.model = model.bind_tools(tools)

  def exists_action(self, state: AgentState):
    result = state['messages'][-1]
    return len(result.tool_calls) > 0

  def call_openai(self, state: AgentState):
    messages = state['messages']
    if self.system:
      messages = [SystemMessage(content=self.system)] + messages
    message = self.model.invoke(messages)
    return {"messages": [message]}
  
  def take_action(self, state: AgentState):
    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
      print(f"Calling: {t}")
      if not t['name'] in self.tools:  # Check for bad tool name from LLM
        print("\n ....bad tool name....")
        result = "bad tool name, retry"  # Instruct LLM to retry if bad
      else:
        result = self.tools[t['name']].invoke(t['args'])
      results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
      print("Back to the model!")
      return {'messages': results}

In [29]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

In [36]:
agent = Agent(llm, [tool], system=prompt)

In [ ]:
%pip install pygraphviz

from IPython.display import Image

Image(agent.graph.get_graph().draw_png())

In [37]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="What is the weather in Campinas, SP?")]
result = agent.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Campinas, SP'}, 'id': 'call_WuWm8GY2XddZQFEzDjSZvAPa', 'type': 'tool_call'}
Back to the model!


In [39]:
print(result['messages'][-1].content)

The current weather in Campinas, SP is as follows:

- **Temperature**: 37.4°C (99.4°F)
- **Condition**: Sunny
- **Wind**: 4.9 mph (7.9 kph) from the East
- **Humidity**: 19%
- **Pressure**: 1011 mb
- **Visibility**: 10 km
- **UV Index**: 10.0 (high)

For more detailed information, you can check [WeatherAPI](https://www.weatherapi.com/) or [Time and Date](https://www.timeanddate.com/weather/brazil/campinas/ext).
